# Basic Weather Agent with MCP Support (StreamableHTTP Transport)

* Collaborator: Roberto Rodriguez @Cyb3rWard0g

## Install Required Libraries
Before starting, ensure the required libraries are installed:

In [ ]:
!pip install dapr-agents python-dotenv mcp starlette

## Import Environment Variables

In [1]:
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.

True

## Enable Logging

In [2]:
import logging

logging.basicConfig(level=logging.INFO)

## Connect to MCP Server and Get Tools

In [3]:
from mcp.client.streamable_http import streamablehttp_client
from mcp import ClientSession

from dapr_agents import Agent, AgentTool

# Connect to the server using SSE
async with streamablehttp_client("http://localhost:8000/mcp/") as (read, write, _):
    async with ClientSession(read, write) as session:
        # Initialize the session
        await session.initialize()
        
        # Get the agent tools from the MCP session
        agent_tools = await AgentTool.from_mcp_session(session)
        
        # Create an agent with the tools
        agent = Agent(
            name="Rob",
            role="Weather Assistant",
            tools=agent_tools
        )

        # Run the agent with a sample query
        await agent.run("What is the weather in Virginia?")

/Users/wardog/Documents/GitHub/dapr-agents/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:mcp.client.streamable_http:Connecting to StreamableHTTP endpoint: http://localhost:8000/mcp/
INFO:httpx:HTTP Request: POST http://localhost:8000/mcp/ "HTTP/1.1 200 OK"
INFO:mcp.client.streamable_http:Received session ID: 79fca29383644485b532e7833aa62994
INFO:httpx:HTTP Request: GET http://localhost:8000/mcp/ "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8000/mcp/ "HTTP/1.1 202 Accepted"
INFO:httpx:HTTP Request: POST http://localhost:8000/mcp/ "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.client.base:Initializing OpenAI client...
INFO:dapr_agents.tool.executor:Tool registered: GetWeather
INFO:dapr_agents.tool.executor:Tool registered: GetHumidity
INFO:dapr_agents.tool.executor:Tool Exe

user:
What is the weather in Virginia?

--------------------------------------------------------------------------------



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.chat:Chat completion retrieved successfully.
INFO:dapr_agents.agents.agent.agent:Executing GetWeather with arguments {'location': 'Virginia'}
INFO:dapr_agents.tool.executor:Running tool (auto): GetWeather
INFO:httpx:HTTP Request: POST http://localhost:8000/mcp/ "HTTP/1.1 200 OK"
INFO:dapr_agents.agents.agent.agent:Iteration 2/10 started.
INFO:dapr_agents.llm.utils.request:Tools are available in the request.
INFO:dapr_agents.llm.openai.chat:Invoking ChatCompletion API.


assistant:
{'content': None, 'role': 'assistant', 'tool_calls': [{'id': 'call_snp9UYuPwlflEm7r0vHM6hIW', 'type': 'function', 'function': {'name': 'GetWeather', 'arguments': '{"location":"Virginia"}'}}]}

--------------------------------------------------------------------------------

GetWeather(tool) (Id: call_snp9UYuPwlflEm7r0vHM6hIW):
Virginia: 69F.

--------------------------------------------------------------------------------



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.chat:Chat completion retrieved successfully.
INFO:httpx:HTTP Request: DELETE http://localhost:8000/mcp/ "HTTP/1.1 200 OK"


assistant:
{'content': 'The current temperature in Virginia is 69°F.', 'role': 'assistant'}

--------------------------------------------------------------------------------



## Connect to StreamableHTTP MCP Servers

In [ ]:
# Initialize MCP Server
# uv run servers/serverA.py --server_type streamable-http
# uv run servers/serverA.py --server_type streamable-http --stateless

In [4]:
# Define your MCP server configs
mcp_servers = {
    "weather": {
        "transport": "streamable_http",
        "url": "http://localhost:8000/mcp/"
    },
}

In [5]:
# Note: The server script (serverA.py) should be running for this to work.
from dapr_agents.tool.mcp.client import MCPClient

# Create an instance of the MCPClient to interact with the MCP servers
client = MCPClient()

In [6]:
# Connect to the MCP servers and get tools and prompts
await client.connect_from_config(mcp_servers)

INFO:mcp.client.streamable_http:Connecting to StreamableHTTP endpoint: http://localhost:8000/mcp/
INFO:httpx:HTTP Request: POST http://localhost:8000/mcp/ "HTTP/1.1 200 OK"
INFO:mcp.client.streamable_http:Received session ID: ce4f94a289954305a3b0da788d7171ed
INFO:httpx:HTTP Request: GET http://localhost:8000/mcp/ "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8000/mcp/ "HTTP/1.1 202 Accepted"
INFO:httpx:HTTP Request: POST http://localhost:8000/mcp/ "HTTP/1.1 200 OK"
INFO:dapr_agents.tool.mcp.client:Loaded 2 tools from server 'weather'
INFO:httpx:HTTP Request: POST http://localhost:8000/mcp/ "HTTP/1.1 200 OK"
INFO:dapr_agents.tool.mcp.client:Loaded 0 prompts from server 'weather': 
INFO:httpx:HTTP Request: DELETE http://localhost:8000/mcp/ "HTTP/1.1 200 OK"
INFO:dapr_agents.tool.mcp.client:Successfully connected to MCP server 'weather' (ephemeral mode)


In [7]:
# Get all tools available
tools = client.get_all_tools()
print("🔧 Tools:", [t.name for t in tools])

🔧 Tools: ['WeatherGetWeather', 'WeatherGetHumidity']


## Initialize Agent

In [8]:
from dapr_agents import Agent

agent = Agent(
    name="Rob",
    role= "Weather Assistant",
    tools=tools
)

INFO:dapr_agents.llm.openai.client.base:Initializing OpenAI client...
INFO:dapr_agents.tool.executor:Tool registered: WeatherGetWeather
INFO:dapr_agents.tool.executor:Tool registered: WeatherGetHumidity
INFO:dapr_agents.tool.executor:Tool Executor initialized with 2 tool(s).
INFO:dapr_agents.agents.base:Constructing system_prompt from agent attributes.
INFO:dapr_agents.agents.base:Using system_prompt to create the prompt template.
INFO:dapr_agents.agents.base:Pre-filled prompt template with attributes: ['name', 'role', 'goal']


## Run Agent

In [9]:
await agent.run("What is the weather in New York?")

INFO:dapr_agents.agents.agent.agent:Iteration 1/10 started.
INFO:dapr_agents.llm.utils.request:Tools are available in the request.
INFO:dapr_agents.llm.openai.chat:Invoking ChatCompletion API.


user:
What is the weather in New York?

--------------------------------------------------------------------------------



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.chat:Chat completion retrieved successfully.
INFO:dapr_agents.agents.agent.agent:Executing WeatherGetWeather with arguments {'location': 'New York'}
INFO:dapr_agents.tool.executor:Running tool (auto): WeatherGetWeather
INFO:mcp.client.streamable_http:Connecting to StreamableHTTP endpoint: http://localhost:8000/mcp/
INFO:httpx:HTTP Request: POST http://localhost:8000/mcp/ "HTTP/1.1 200 OK"
INFO:mcp.client.streamable_http:Received session ID: d5a91177c5fb4102b175b4b5dbb9e1ba
INFO:httpx:HTTP Request: POST http://localhost:8000/mcp/ "HTTP/1.1 202 Accepted"
INFO:httpx:HTTP Request: GET http://localhost:8000/mcp/ "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8000/mcp/ "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: DELETE http://localhost:8000/mcp/ "HTTP/1.1 200 OK"
INFO:dapr_agents.agents.agent.agent:Iteration 2/10 started.
INFO:dapr_agents.llm.utils.request:T

assistant:
{'content': None, 'role': 'assistant', 'tool_calls': [{'id': 'call_uTrpRZnEP4JUt0yMJhnPQoQ0', 'type': 'function', 'function': {'name': 'WeatherGetWeather', 'arguments': '{"location":"New York"}'}}]}

--------------------------------------------------------------------------------

WeatherGetWeather(tool) (Id: call_uTrpRZnEP4JUt0yMJhnPQoQ0):
New York: 80F.

--------------------------------------------------------------------------------



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.chat:Chat completion retrieved successfully.


assistant:
{'content': 'The current weather in New York is 80°F.', 'role': 'assistant'}

--------------------------------------------------------------------------------



'The current weather in New York is 80°F.'